In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import pickle

In [2]:
# make sure pandas is version 1.0 or higher
# make sure networkx is verion 2.4 or higher
print(pd.__version__)
print(nx.__version__)

2.1.3
3.2.1


In [3]:
from ema_workbench import (
    Model,
    Policy,
    ema_logging,
    SequentialEvaluator,
    MultiprocessingEvaluator, 
    perform_experiments,
    Samplers, 
)
from SALib.analyze import sobol
from ema_workbench.em_framework.salib_samplers import get_SALib_problem
from ema_workbench.analysis import (feature_scoring, pairs_plotting)

from dike_model_function import DikeNetwork  # @UnresolvedImport
from problem_formulation import get_model_for_problem_formulation, sum_over, sum_over_time



In [4]:
ema_logging.log_to_stderr(ema_logging.INFO)

# choose problem formulation number, between 0-5
# each problem formulation has its own list of outcomes
dike_model, planning_steps = get_model_for_problem_formulation(2)

In [5]:
# enlisting uncertainties, their types (RealParameter/IntegerParameter/CategoricalParameter), lower boundary, and upper boundary
import copy

for unc in dike_model.uncertainties:
    print(repr(unc))

uncertainties = copy.deepcopy(dike_model.uncertainties)

CategoricalParameter('discount rate 0', [0, 1, 2, 3])
CategoricalParameter('discount rate 1', [0, 1, 2, 3])
CategoricalParameter('discount rate 2', [0, 1, 2, 3])
IntegerParameter('A.0_ID flood wave shape', 0, 132, resolution=None, default=None, variable_name=['A.0_ID flood wave shape'], pff=False)
RealParameter('A.1_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.1_Bmax'], pff=False)
RealParameter('A.1_pfail', 0, 1, resolution=None, default=None, variable_name=['A.1_pfail'], pff=False)
CategoricalParameter('A.1_Brate', [0, 1, 2])
RealParameter('A.2_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.2_Bmax'], pff=False)
RealParameter('A.2_pfail', 0, 1, resolution=None, default=None, variable_name=['A.2_pfail'], pff=False)
CategoricalParameter('A.2_Brate', [0, 1, 2])
RealParameter('A.3_Bmax', 30, 350, resolution=None, default=None, variable_name=['A.3_Bmax'], pff=False)
RealParameter('A.3_pfail', 0, 1, resolution=None, default=None, variable_name=['A.3_pfai

In [6]:
# enlisting policy levers, their types (RealParameter/IntegerParameter), lower boundary, and upper boundary
for policy in dike_model.levers:
    print(repr(policy))

levers = copy.deepcopy(dike_model.levers)

IntegerParameter('0_RfR 0', 0, 1, resolution=None, default=None, variable_name=['0_RfR 0'], pff=False)
IntegerParameter('0_RfR 1', 0, 1, resolution=None, default=None, variable_name=['0_RfR 1'], pff=False)
IntegerParameter('0_RfR 2', 0, 1, resolution=None, default=None, variable_name=['0_RfR 2'], pff=False)
IntegerParameter('1_RfR 0', 0, 1, resolution=None, default=None, variable_name=['1_RfR 0'], pff=False)
IntegerParameter('1_RfR 1', 0, 1, resolution=None, default=None, variable_name=['1_RfR 1'], pff=False)
IntegerParameter('1_RfR 2', 0, 1, resolution=None, default=None, variable_name=['1_RfR 2'], pff=False)
IntegerParameter('2_RfR 0', 0, 1, resolution=None, default=None, variable_name=['2_RfR 0'], pff=False)
IntegerParameter('2_RfR 1', 0, 1, resolution=None, default=None, variable_name=['2_RfR 1'], pff=False)
IntegerParameter('2_RfR 2', 0, 1, resolution=None, default=None, variable_name=['2_RfR 2'], pff=False)
IntegerParameter('3_RfR 0', 0, 1, resolution=None, default=None, variable

In [7]:
# enlisting outcomes
for outcome in dike_model.outcomes:
    print(repr(outcome))

ScalarOutcome('Expected Annual Damage', variable_name=('A.1_Expected Annual Damage', 'A.2_Expected Annual Damage', 'A.3_Expected Annual Damage', 'A.4_Expected Annual Damage', 'A.5_Expected Annual Damage'), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Dike Investment Costs', variable_name=('A.1_Dike Investment Costs', 'A.2_Dike Investment Costs', 'A.3_Dike Investment Costs', 'A.4_Dike Investment Costs', 'A.5_Dike Investment Costs'), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('RfR Investment Costs', variable_name=('RfR Total Costs',), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Evacuation Costs', variable_name=('Expected Evacuation Costs',), function=<function sum_over at 0x0000020045F561F0>)
ScalarOutcome('Expected Number of Deaths', variable_name=('A.1_Expected Number of Deaths', 'A.2_Expected Number of Deaths', 'A.3_Expected Number of Deaths', 'A.4_Expected Number of Deaths', 'A.5_Expected Number of Deaths'), function

# Putting the outcomes in a dataframe

In [13]:
with open('saved_experiments.pkl', 'rb') as file:
    experiments = pickle.load(file)
    
with open('saved_outcomes.pkl', 'rb') as file:
    outcomes = pickle.load(file)

In [51]:
# Store all outcome names in a list
outcome_names = list(outcomes.keys())
print(outcome_names)


['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths', 'A.4 Total Costs', 'A.4_Expected Number of Deaths', 'A.5 Total Costs', 'A.5_Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']


In [79]:
# Make a Dataframe with the Outcomes on the columns and the Uncertainties as rows
Si = sobol.analyze(problem, outcomes["A.1 Total Costs"], calc_second_order=True, print_to_console=True)

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))


                               ST   ST_conf
A.0_ID flood wave shape  0.001555  0.000613
A.1_Bmax                 0.000000  0.000000
A.1_Brate                0.000000  0.000000
A.1_pfail                0.985283  0.073087
A.2_Bmax                 0.000000  0.000000
A.2_Brate                0.000000  0.000000
A.2_pfail                0.000000  0.000000
A.3_Bmax                 0.000000  0.000000
A.3_Brate                0.000000  0.000000
A.3_pfail                0.000000  0.000000
A.4_Bmax                 0.000000  0.000000
A.4_Brate                0.000000  0.000000
A.4_pfail                0.000000  0.000000
A.5_Bmax                 0.000000  0.000000
A.5_Brate                0.000000  0.000000
A.5_pfail                0.000000  0.000000
discount rate 0          0.012768  0.001763
discount rate 1          0.012664  0.001550
discount rate 2          0.012649  0.001751
                               S1   S1_conf
A.0_ID flood wave shape -0.002125  0.003911
A.1_Bmax                 0.00000

In [80]:
Si_filter = {k:Si[k] for k in ['ST','ST_conf']}
Si_df = pd.DataFrame(Si_filter, index=problem['names'])

,ST,ST_conf,S1,S1_conf
A.0_ID flood wave shape,0.001555,0.000613,-0.002125,0.003911
A.1_Bmax,0.000000,0.000000,0.000000,0.000000
A.1_Brate,0.000000,0.000000,0.000000,0.000000
A.1_pfail,0.985283,0.073087,0.957308,0.077135
A.2_Bmax,0.000000,0.000000,0.000000,0.000000
A.2_Brate,0.000000,0.000000,0.000000,0.000000
A.2_pfail,0.000000,0.000000,0.000000,0.000000
A.3_Bmax,0.000000,0.000000,0.000000,0.000000
A.3_Brate,0.000000,0.000000,0.000000,0.000000
A.3_pfail,0.000000,0.000000,0.000000,0.000000


In [84]:
results_dict = {}

for outcome in outcome_names:
    # Perform the Sobol analysis for each outcome
    Si = sobol.analyze(problem, outcomes[outcome], calc_second_order=True, print_to_console=False)
    
    # Filter the results
    Si_filter = {k: Si[k] for k in ['ST', 'ST_conf']}
    
    # Create a DataFrame from the filtered results
    Si_df = pd.DataFrame(Si_filter, index=problem['names'])
    
    # Store the DataFrame in the results dictionary
    results_dict[outcome] = Si_df

# Concatenate all the DataFrames in the results dictionary into a single DataFrame
# Use MultiIndex to handle multiple outcomes
final_df = pd.concat(results_dict, axis=1)

# Display the final DataFrame
display(final_df)

C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\util\__init__.py:274: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  names = list(pd.unique(groups))
C:\Users\mmtud\anaconda3\envs\gds\lib\site-packages\SALib\analyze\sobol.py:141: RuntimeWarning: invalid value encountered in divide
  Y = (Y - Y.mean()) / Y.std()


A.1 Total Costs            \
                                     ST   ST_conf   
A.0_ID flood wave shape        0.001555  0.000599   
A.1_Bmax                       0.000000  0.000000   
A.1_Brate                      0.000000  0.000000   
A.1_pfail                      0.985283  0.075174   
A.2_Bmax                       0.000000  0.000000   
A.2_Brate                      0.000000  0.000000   
A.2_pfail                      0.000000  0.000000   
A.3_Bmax                       0.000000  0.000000   
A.3_Brate                      0.000000  0.000000   
A.3_pfail                      0.000000  0.000000   
A.4_Bmax                       0.000000  0.000000   
A.4_Brate                      0.000000  0.000000   
A.4_pfail                      0.000000  0.000000   
A.5_Bmax                       0.000000  0.000000   
A.5_Brate                      0.000000  0.000000   
A.5_pfail                      0.000000  0.000000   
discount rate 0                0.012768  0.001578   
discount rate 1                0.012664  0.001830   
discount rate 2                0.012649  0.001765   

                        A.1_Expected Number of Deaths            \
                                                   ST   ST_conf   
A.0_ID flood wave shape                      0.001815  0.000691   
A.1_Bmax                                     0.000000  0.000000   
A.1_Brate                                    0.000000  0.000000   
A.1_pfail                                    0.999837  0.065206   
A.2_Bmax                                     0.000000  0.000000   
A.2_Brate                                    0.000000  0.000000   
A.2_pfail                                    0.000000  0.000000   
A.3_Bmax                                     0.000000  0.000000   
A.3_Brate                                    0.000000  0.000000   
A.3_pfail                                    0.000000  0.000000   
A.4_Bmax                                     0.000000  0.000000   
A.4_Brate                                    0.000000  0.000000   
A.4_pfail                                    0.000000  0.000000   
A.5_Bmax                                     0.000000  0.000000   
A.5_Brate                                    0.000000  0.000000   
A.5_pfail                                    0.000000  0.000000   
discount rate 0                              0.000000  0.000000   
discount rate 1                              0.000000  0.000000   
discount rate 2                              0.000000  0.000000   

                        A.2 Total Costs            \
                                     ST   ST_conf   
A.0_ID flood wave shape        0.005098  0.001949   
A.1_Bmax                       0.023350  0.014618   
A.1_Brate                      0.000099  0.000130   
A.1_pfail                      0.201903  0.055167   
A.2_Bmax                       0.000000  0.000000   
A.2_Brate                      0.000000  0.000000   
A.2_pfail                      0.899929  0.085159   
A.3_Bmax                       0.000000  0.000000   
A.3_Brate                      0.000000  0.000000   
A.3_pfail                      0.000000  0.000000   
A.4_Bmax                       0.000000  0.000000   
A.4_Brate                      0.000000  0.000000   
A.4_pfail                      0.000000  0.000000   
A.5_Bmax                       0.000000  0.000000   
A.5_Brate                      0.000000  0.000000   
A.5_pfail                      0.000000  0.000000   
discount rate 0                0.009626  0.002269   
discount rate 1                0.009510  0.001981   
discount rate 2                0.009031  0.002017   

                        A.2_Expected Number of Deaths            \
                                                   ST   ST_conf   
A.0_ID flood wave shape                      0.004930  0.001572   
A.1_Bmax                                     0.025907  0.012533   
A.1_Brate                                    0.000147  0.000198   
A.1_pfail                                    0.216402

In [85]:
final_df.columns

MultiIndex([(              'A.1 Total Costs',      'ST'),
            (              'A.1 Total Costs', 'ST_conf'),
            ('A.1_Expected Number of Deaths',      'ST'),
            ('A.1_Expected Number of Deaths', 'ST_conf'),
            (              'A.2 Total Costs',      'ST'),
            (              'A.2 Total Costs', 'ST_conf'),
            ('A.2_Expected Number of Deaths',      'ST'),
            ('A.2_Expected Number of Deaths', 'ST_conf'),
            (              'A.3 Total Costs',      'ST'),
            (              'A.3 Total Costs', 'ST_conf'),
            ('A.3_Expected Number of Deaths',      'ST'),
            ('A.3_Expected Number of Deaths', 'ST_conf'),
            (              'A.4 Total Costs',      'ST'),
            (              'A.4 Total Costs', 'ST_conf'),
            ('A.4_Expected Number of Deaths',      'ST'),
            ('A.4_Expected Number of Deaths', 'ST_conf'),
            (              'A.5 Total Costs',      'ST'),
            ( 

# Analysis of the SOBAL SA outcomes